In [ ]:
# from Andrej Karpathy's tutorial on ChatGPT - a tidied up notebook of the initial code

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-03 18:32:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2023-06-03 18:32:47 (115 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
batch_size = 32 # how many independent seqeunces will we process in parallel?
block_size = 32 # what is the maximum context length for predictions? 
n_embed = 32

training_iterations = 5000
evaluation_interval = 500 #validation every evaluation_interval steps of optimsation.
# take average of evaluation_batches batches for loss calculations 
#  in the function estimate_loss, to reduce the noise in the estimates.
evaluation_batches = 20 
learning_rate = 1e-3
# device agnostic definition
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split): # data loader
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  #randomly pick starts for batch_size number of smaples and then loop through
  # batch samples, stacking each sample in a row to give a batch * block_size tensor
  ix = torch.randint(len(data) - block_size, (batch_size,)) 
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1: i+block_size+1] for i in ix])
  x.to(device), y.to(device)
  return x, y 

# below function new code compared to what was in the Jupyter notebook
# used for sketching out details. 
@torch.no_grad()
def estimate_loss(eval_batches):
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_batches)
    for k in range(eval_batches):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class FeedForward(nn.Module):
    
    def __init__(self, n_embed):
      super().__init__()
      self.ff = nn.Sequential(
          nn.Linear(n_embed, 4*n_embed),
          nn.ReLU(),
          nn.Linear(4*n_embed, n_embed)
      )

    def forward(self, x):
      return self.ff(x)


class Head(nn.Module):
  """one head of self-attention"""
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias = False) # (B, T, 16)
    self.query = nn.Linear(n_embed, head_size, bias=False) # (B, T, 16)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    # lower triangular matrix, below is not needed for encoder, but decoder needs
    # past not to see the future, so needs masking.
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self,x):
    B, T, C = x.shape # batch, time, channels
    # Each token learns during training to give vector that is a query
    # and one that is a key. 
    # These represent 
    # Query - what looking for? What in my local environment is interesting? 
    # Key - what do I contain? What do I have that another token might need?
    # query dot key indicates how well these two match.
    # value is the learnt value that should be emitted for a match.
    k = self.key(x)
    q = self.query(x)
    v = self.value(x)
    weights = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)
    weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # not needed for encoder
    weights = F.softmax(weights, dim=-1)
    out = weights @ v 
    return out
# attention is a communication mechanism. Take the information and transform it.
# no notion of space therefore need to code positional information. 
# batches are independent of each other.
# this is a decoder so past cannot see the future. Encoder head does not have
# this restriction and removes the lower triangular matrix filter


class MultiHeadAttention(nn.Module):
  '''multiple heads of self-attention in parallel'''
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed, n_embed) 

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    return self.proj(out)

class TransformerBlock(nn.Module):
  ''' pass input through multiheaded attention and then feed forward mlp. '''
  # transformer block - communication followed by multilayer perceptron 
  # communication.
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.layer_norm1 = nn.LayerNorm(n_embed)
    self.sa = MultiHeadAttention(n_head, head_size)
    self.layer_norm2 = nn.LayerNorm(n_embed)
    self.fforward = FeedForward(n_embed)

  def forward(self, x):
    x = self.layer_norm1(x)
    x = x + self.sa(x)
    x = self.layer_norm2(x)
    x = x + self.fforward(x)
    return x



class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # basic bigram read each token directly reads off the logits for the next 
    # token from a lookup table, so could be interpretted as a probability via 
    # a softmax layer, 
    # but now the embedings will not be the probability of the next character,
    # being instead a latent representation that will get processed through 
    # further layers before producing the logits that feed into a softmax.
    # The embedding table is thus now of size vocab_size * n_embed
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    #self.sa_heads = MultiHeadAttention(4, n_embed//4)
    #self.fforward = FeedForward(n_embed)
    # replace above two entries with a transformer block containing both
    self.transformer_blocks = nn.Sequential(
        TransformerBlock(n_embed, n_head=4),
        TransformerBlock(n_embed, n_head=4),
        TransformerBlock(n_embed, n_head=4),
        nn.LayerNorm(n_embed)
    )
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    # nn module of pytorch, which this inherits from runs forward when called
    # e.g. below as 
    # m = BigramLanguageModel(vocab_size)
    # logits, loss = m(xb, yb)
    # idx - (index of) the x values, i.e. the context vector
    # idx and targets are both (B, T) tensor of integers (see comment below)
    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb # (B, T, C)
    #x = self.sa_heads(x1)
    #x = self.fforward(x)
    #x = x + x1
    # replace above two lines with transformer blocks, defined at instantiation
    x = self.transformer_blocks(x)
    logits = self.lm_head(x) # (B, T, vocab_size)
    # B T C is batch by time by channel
    # batch  is 4
    # time is 8 (block size)
    # channel is 65 i.e. vocab size i.e. which element of vocab
    # with row of embedding table representing likely next character of the sequence.
    # Summary: The embedding table receives a tensor, idx, of dimension B*T and 
    # for each entry returns a dimension of size C.
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape # need to reshape to be correct shape for cross_entropy
      # use view in pytorch that changes the view of the data passed but not the
      # underlying data. Flatten the first two dimensions, leaving the channel (vocab)
      # as second dimension, which is what F.cross_entropy expects.
      # print(f"Targets {targets} \n; logits {logits} ")
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss
  
  def generate(self, idx, max_new_tokens):
    '''Predict the next character/token from the learnt distribution,adding
    it to the current context, idx, till
    max_new_tokens have been added.'''
    for _ in range(max_new_tokens):
      #get the predictions
      # crop idx to the last block_size number of tokens, which is needed
      # since the attention layer has positional embedding only up to length
      # block size
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond) # calls forwards() for this class
      # next line gets the last time step, i.e. the last character for 
      # each sequence in the
      # batch, taking all batches and all embeddings for that character, and place
      # in logits.
      logits = logits[:, -1, :] # when we develop this later we'll focus on more 
      # context but currently only have bigram model, so just taking last 
      # token/logit.
      # so logits is now dimension B, C, since only one token
      # generate probability distribution
      probs = F.softmax(logits, dim=-1) # B, C
      # sample from the vocab according to the probability distribution encoded
      # by the logits
      idx_next = torch.multinomial(probs, num_samples = 1) # B by 1 size
      # append sample to sequence
      idx = torch.cat((idx, idx_next), dim = 1)
    return idx

text = open('input.txt','r',encoding='utf-8').read()
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers and reverse
stoi = { ch:i for i, ch in enumerate(chars)}  #dictionary with key character, token i
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encode string as integers
decode = lambda l: ''.join([itos[i] for i in l]) #decode list of integers as output string
data = torch.tensor(encode(text), dtype = torch.long)

#simple train /validation sets. No test set. no randomisation of selections, so assuming
# no bias in the distribution within the data file!
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
x = train_data[:block_size]
y = train_data[1:block_size + 1] # we're training to predict the next character.

xb, yb = get_batch('train')

model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
print(f"untrained loss: {loss}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("Text generated from untrained network: ")
print(decode(m.generate(context, max_new_tokens = 100)[0].tolist()))

# training loop. 
# high learning rate since simple network
print(f"training for {training_iterations} iterations")
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate) 

for iter in range(training_iterations):
  xb, yb = get_batch('train')
  #xval, yval = get_batch('validation')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  #equally uncertain distribution implies 1/vocab_size = 1/65 probability of each 
  # next character which implies -ln(1/65) = 4.17 would be optimal initial loss
  if iter % evaluation_interval == 0:
    #logits_val, loss_val = m(xval, yval)
    #print(f"training loss: {loss.item()}, \
    #                                       validation loss {loss_val.item()}")
    losses = estimate_loss(evaluation_batches)
    print(f"step {iter}: train loss {losses['train']:.4f}, \
                                         validation loss {losses['val']:.4f})")
print(f"Text from trained network, trained {training_iterations} iterations:")
print(decode(m.generate(context, max_new_tokens = 100)[0].tolist()))

untrained loss: 4.33352518081665
Text generated from untrained network: 

FHcJksEx
VRHA3Z?CLY Xkg
FAr;G'z-DY,Gr&j;IU&xXLMWl,Y?GMclS-JLjlKkTXT,sNQNq&xYd$bMftRGSLDiYXCmJIbPBMPt
training for 5000 iterations
step 0: train loss 4.2323,                                          validation loss 4.2459)
step 500: train loss 2.4130,                                          validation loss 2.4179)
step 1000: train loss 2.2572,                                          validation loss 2.2725)
step 1500: train loss 2.1664,                                          validation loss 2.1889)
step 2000: train loss 2.0823,                                          validation loss 2.1343)
step 2500: train loss 2.0130,                                          validation loss 2.0842)
step 3000: train loss 1.9974,                                          validation loss 2.0700)
step 3500: train loss 1.9662,                                          validation loss 2.0457)
step 4000: train loss 1.9458,           

Below just playing with batch norm and layer norm. Not used in the above code. 

In [ ]:
class BatchNorm:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    #parameters trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = toch.zeros(dim)
    #buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

    def __call__(self, x):
      #calculate forward pass
      if self.training: 
        xmean = x.mean(0, keepdim=True) # batch mean
        xvar = x.var(0, keepdim=True) # batch variance
      else:
        xmean = self.running_mean
        xvar = self.running_var
      xhat = (x - xmean) /toch.sqrt(xvar + self.eps) # normalize to unit variance
      self.out = self.gamma * xhat + self.beta
      # update the buffers
      if self.training:
        with torch.no_grad():
          self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
          self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
      return self.out

    def parameters(self):
      return [self.gamma, self.beta]

In [ ]:
class LayerNorm:
  def __init__(self, dim, eps=1e-5):
    self.eps = eps
    #parameters trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)


    def __call__(self, x):
      #calculate forward pass
      xmean = x.mean(1, keepdim=True) #  mean of layer
      xvar = x.var(1, keepdim=True) #  variance of layer
      xhat = (x - xmean) /toch.sqrt(xvar + self.eps) # normalize to unit variance
      self.out = self.gamma * xhat + self.beta
      return self.out

    def parameters(self):
      return [self.gamma, self.beta]

In [ ]:
import torch 
#a = torch.randn(4,4)
a = torch.ones(4, 3)
a


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
a[1,1]=3
a.mean(1, keepdim=True)

tensor([[1.0000],
        [1.6667],
        [1.0000],
        [1.0000]])

In [ ]:
a.mean(0, keepdim=True)

tensor([[1.0000, 1.5000, 1.0000]])